In [15]:
''' This cell contains all constants thay may differ on our machines '''

ELASTIC_HOST = 'localhost'
ELASTIC_PORT = 9200
COLLECTION_DIRECTORY = "../byweb" # directory with .out files to process
COLLECTION_DIRECTORY_MYSTEM = "../byweb_stem" # directory with .out files after mystem processing

QUERIES_FILE = "../web2008_adhoc.xml"
RELEVANCE_FILE = "../relevant_table_2009.xml"

In [61]:
class Document:
    def __init__(self, doc_url, doc_id, sz_bytes, sz_words):
        self.url = doc_url       # document url
        self.id = doc_id         # unique document id (str)
        self.sz_bytes = sz_bytes # document size in bytes before deleting html markup
        self.sz_words = sz_words # number of words in document before deleting html markup
        self.words = []          # list of words in document after deleting html markup
        self.links = []          # lisk of links in document

In [62]:
import os
from tqdm import tqdm
from tqdm import tqdm_notebook
import pickle

class BaseDocumentProcessor:
    def process(self, document, title):
        pass
    def result(self):
        pass

def process_file(d, f, processor, pbar):
    print("processing", os.path.join(d, f))
    with open(os.path.join(d, f), "rb") as fin, open(os.path.join(d, f.replace(".out", ".title")), "rb") as tfin:
        dct = pickle.load(tfin)
        while True:
            pbar.update(1)
            try:
                document = pickle.load(fin)
            except:
                break
            processor.process(document, dct[document.id])

def process_collection(directory, processor):
    pbar = tqdm(total = 200000)
    for file in os.listdir(directory):
        if (file.endswith(".out")):
            process_file(directory, file, processor, pbar)

In [7]:
from pymystem3 import Mystem

def stem_queries(queries):
    m = Mystem(grammar_info=False, disambiguation=False)
    reg = re.compile('[a-zа-яё0-9\-]')
    filterFunc = lambda w : reg.match(w)
    result = {}
    for (qid, text) in queries.items():
        result[qid] = ' '.join(filter(filterFunc, m.lemmatize(text)))
    return result

In [58]:
from rank_bm25 import BM25Okapi #https://pypi.org/project/rank-bm25/ examples
from rank_bm25 import BM25Plus

def __feature_bm25(documents, queries, BM):
    all_words = [doc.words for doc in documents]
    bm25 = BM(all_words)
    return [bm25.get_scores(q.split(" ")) for q in queries]

def feature_bm25Plus(documents, queries):
    return __feature_bm25(documents, queries, BM25Plus)

def feature_bm25(documents, queries):
    return __feature_bm25(documents, queries, BM25Okapi)

In [45]:
class QWord:
    def __init__(self, word, pos, prev = None):
        self.prev = prev
        self.next = None
        self.word = word
        self.pos = pos
        self.step = 0
        if (prev != None):
            prev.next = self
            self.step = pos - prev.pos

def check(d):
    for k, v in d.items():
        if v == 0:
            return False
    return True
        
def feature_window(document, query, title):
    queryset = set(query.split(' '))
    head = None
    tail = None
    for i, w in enumerate(document.words):
        if w in queryset:
            if head == None:
                head = QWord(w, i)
                tail = head
            else:
                tail = QWord(w, i, tail)
                
    collect = dict.fromkeys(queryset, 0)
    node1 = head
    node2 = head
    collect[head.word] = 1
    length = 0
    minLength = len(document.words)
    
    while node1.next != None:
        node1 = node1.next
        collect[node1.word] += 1
        length += node1.step
        while node2.next != None and check(collect):
            minLength = min(minLength, length)
            collect[node2.word] -= 1
            node2 = node2.next
            length -= node2.step
    return minLength

In [54]:
def feature_query_len(document, query, title):
    return len(query)

def feature_query_list_len(document, query, title):
    return len(query.split(' '))

def featrue_doc_len(docuemnt, query, title):
    return len(document.words)

In [63]:
class FeatureProcessor(BaseDocumentProcessor):
    def __init__(self, features):
        self.features = features
    
    def process(self, document, title):
        [for f in self.feautures]
        return
        
    def result(self):
        pass